In [1]:
from src.briscola_env.Briscola import Briscola

env = Briscola()

In [2]:
env.step(0)
print(env.game_state)

State(deck=[32, 22, 28, 0, 6, 31, 34, 20, 33, 7, 14, 12, 10, 16, 17, 21, 1, 36, 3, 30, 18, 38, 25, 15, 9, 13, 26, 2, 39, 11, 5, 19, 4, 24], seen_cards=[], hand_cards={'player_0': [29, 23], 'player_1': [37, 8, 35]}, table_card=27, briscola_card=32, current_agent='player_1', agent_points={'player_0': 0, 'player_1': 0})


In [3]:
env.terminations

{'player_0': False, 'player_1': False}

In [5]:
env.agents

['player_1']

In [5]:
env.agent_selection = 3

AttributeError: can't set attribute